In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd
import mlflow

/Users/christelleren/DSS/design/code-envs/python/py36_mlflow/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/Users/christelleren/DSS/design/code-envs/python/py36_mlflow/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: MLflow support for Python 3.6 is deprecated and will be dropped in an upcoming release. At that point, existing Python 3.6 workflows that use MLflow will continue to work without modification, but Python 3.6 users will no longer get access to the latest MLflow features and bugfixes. We recommend that you upgrade to Python 3.7 or newer.
  after removing the cwd from sys.path.


In [3]:
from dss_mlflow import preprocessing

In [4]:
from sklearn.linear_model import ElasticNet

In [5]:
project = dataiku.api_client().get_default_project()
managed_folder = project.get_managed_folder('8ip4v4B7')

In [6]:
# Read recipe inputs
train_dataset = dataiku.Dataset("training_data")
df = train_dataset.get_dataframe()

In [9]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

alpha = 0.5
l1_ratio = 0.5

In [10]:
X, y = preprocessing.clean_df(df)

In [16]:
with project.setup_mlflow(managed_folder=managed_folder) as mlflow:

    # Note: if you don't call this (i.e. when no experiment is specified), the default one is used
    mlflow.set_experiment("My first experiment")

    with mlflow.start_run(run_name="my_run"):
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(X, y)

        predicted_qualities = lr.predict(X)

        (rmse, mae, r2) = eval_metrics(y, predicted_qualities)

        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        #mlflow.log_metric("r2", r2)
        #mlflow.log_metric("mae", mae)
        mlflow.log_param("a", 1)
        mlflow.log_metric("b", 2)
        mlflow.sklearn.log_model(lr, "model")

MlflowException: Cannot set a deleted experiment 'My first experiment' as the active experiment. You can restore the experiment, or permanently delete the  experiment to create a new one.